# CS246 - Colab 5
## PageRank

### Setup

First of all, we authenticate a Google Drive client to download the dataset we will be processing in this Colab.

**Make sure to follow the interactive instructions.**

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
id='1EoolSK32_U74I4FeLox88iuUB_SUUYsI'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('web-Stanford.txt')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loading

For this Colab we will be using [NetworkX](https://networkx.github.io), a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.

The dataset we will analyze is a snapshot of the Web Graph centered around [stanford.edu](https://stanford.edu), collected in 2002. Nodes represent pages from Stanford University (stanford.edu) and directed edges represent hyperlinks between them. [[More Info]](http://snap.stanford.edu/data/web-Stanford.html)

In [4]:
import networkx as nx

G = nx.read_edgelist('web-Stanford.txt', create_using=nx.DiGraph)

In [5]:
print(nx.info(G))

DiGraph with 281903 nodes and 2312497 edges


### Your Task

To begin with, let's simplify our analysis by ignoring the dangling nodes and the disconnected components in the original graph.

Use NetworkX to identify the **largest** weakly connected component in the ```G``` graph.  From now on, use this connected component for all the following tasks.

In [6]:
# YOUR CODE HERE
largest_len = 0
for c in sorted(nx.weakly_connected_components(G), key=len, reverse=True):
  if len(c) >= largest_len:
    largest_len = len(c)
    largest_component = c

len(largest_component)

255265

Compute the PageRank vector, using the default parameters in NetworkX: [https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pageranky](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pagerank)

In [7]:
# YOUR CODE HERE
G_sub = G.subgraph(largest_component)
pr = nx.pagerank(G_sub)
len(pr)

255265

In 1999, Barabási and Albert proposed an elegant mathematical model which can generate graphs with topological properties similar to the Web Graph (also called Scale-free Networks).

If you complete the steps below, you should obtain some empirical evidence that the Random Graph model is inferior compared to the Barabási–Albert model when it comes to generating a graph resembling the World Wide Web!

As such, we will use two different graph generator methods, and then we will test how well they approximate the Web Graph structure by means of comparing the respective PageRank vectors. [[NetworkX Graph generators]](https://networkx.github.io/documentation/stable/reference/generators.html#)

Using the parameters ```seed = 1``` and ```directed=False``` where applicable, generate:


1.   a random graph (with the fast method), setting ```n``` equal to the number of nodes in the original connected component, and ```p = 0.00008```
2.   a Barabasi-Albert graph (with the standard method), setting ```n``` equal to the number of nodes in the original connected component, and finding the right ***integer*** value for ```m``` such as the resulting number of edges **approximates by excess** the number of edges in the original connected component

and compute the PageRank vectors for both graphs.


In [8]:
## Finding the # of edges in the original connected component.
from tqdm import tqdm
count = 0
for u, v in tqdm(G.edges()):
  if u in largest_component and v in largest_component:
    count += 1

count

100%|██████████| 2312497/2312497 [00:01<00:00, 1556404.29it/s]


2234572

There are 2234572 edges in the largest component.

In [9]:
# YOUR CODE HERE
G_ER_Random = nx.fast_gnp_random_graph(255265, p=0.00008, seed=1, directed=False)
G_BA = nx.barabasi_albert_graph(255265, 9, seed=1)
len(G_BA.edges)

2297304

When m=9, len(edges) = 2297304

Compare the PageRank vectors obtained on the generated graphs with the PageRank vector you computed on the original connected component.
**Sort** the components of each vector by value, and use cosine similarity as similarity measure. 

Feel free to use any implementation of the cosine similarity available in third-party libraries, or implement your own with ```numpy```.

In [10]:
# YOUR CODE HERE
import math
def cos_sim(u, v):
  
  u_norm, v_norm, sum = 0, 0, 0
  for i in range(len(u)):
    try:
      u_norm += u[i]*u[i]
      v_norm += v[i]*v[i]
      sum += v[i]*u[i]
    except IndexError:
      print(i)
  u_norm = math.sqrt(u_norm)
  v_norm = math.sqrt(v_norm)
  return sum/(u_norm*v_norm)

print("Calculating the PageRank vectors")
pr_ER_Random = nx.pagerank(G_ER_Random)
pr_G_BA = nx.pagerank(G_BA)

##print(len(pr_ER_Random), len(pr_G_BA))
assert len(pr_ER_Random) == len(pr_G_BA)
pr_list = [pr[v] for v in pr]
pr_ER_Random_list = [pr_ER_Random[v] for v in pr_ER_Random]
pr_G_BA_list = [pr_G_BA[v] for v in pr_G_BA]

pr_list.sort()
pr_ER_Random_list.sort()
pr_G_BA_list.sort()

cos_sim(pr_list, pr_ER_Random_list), cos_sim(pr_list, pr_G_BA_list)

Calculating the PageRank vectors


(0.10395564703964517, 0.6488673660967251)

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!